In [ ]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from torch.nn.functional import cosine_similarity

In [ ]:
embedder = SentenceTransformer('sentence-transformers/LaBSE', device='cuda')
original_filename = "../../Data/pipeline_df_with_features.pickle"
updated_filename = "../../Data/pipeline_df_with_features_updated.pickle"

if os.path.exists(updated_filename):
    df = pd.read_pickle(updated_filename)
else:
    df = pd.read_pickle(original_filename)
    batch_size = 512
    source_embeddings = embedder.encode(
        df['fr'].tolist(),
        convert_to_tensor=True,
        batch_size=batch_size,
        device='cuda'
    )
    translated_embeddings = embedder.encode(
        df['en'].tolist(),
        convert_to_tensor=True,
        batch_size=batch_size,
        device='cuda'
    )
    similarities = cosine_similarity(source_embeddings, translated_embeddings)
    df['similarity_update'] = similarities.cpu().numpy()

    df.to_pickle(updated_filename)

In [ ]:
percentiles = [0.00135, 0.02275, 0.1587, 0.8413, 0.97725, 0.99865]
rows = ['0.1%', '2.3%', '15.9%', '50%', '84.1%', '97.7%', '99.9%']
columns = ["similarity", "similarity_update"]

df[columns].describe(percentiles).loc[rows]